In [ ]:
import pandas as pd


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving scraped_zameen_data.xlsx - Sheet1.csv to scraped_zameen_data.xlsx - Sheet1.csv


In [ ]:
import os


In [ ]:
print(os.listdir())



['.config', 'scraped_zameen_data.xlsx - Sheet1.csv', 'sample_data']


In [ ]:
import pandas as pd
df = pd.read_csv("scraped_zameen_data.xlsx - Sheet1.csv", nrows=5)
print(df.head())
print(df.columns.tolist())





In [ ]:
important_cols = ['City', 'Location', 'Price', 'Type', 'Area', 'Bedrooms', 'Bathrooms', 'Kitchens', 'Description']


In [ ]:
import pandas as pd

df_small = pd.read_csv("scraped_zameen_data.xlsx - Sheet1.csv")


In [ ]:
print(df_small.columns.tolist())
df_small.head()


**FOR MISSING VALUES **

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_small = pd.read_csv("scraped_zameen_data.xlsx - Sheet1.csv",
                       usecols=['City', 'Type', 'Area', 'Price', 'Location', 'Description', 'Bedrooms', 'Bathrooms', 'Kitchens'])

In [ ]:
print(df_small.head())

In [ ]:
print(df_small.info())

In [ ]:
print(df_small.describe())

In [ ]:
print(df_small.isnull().sum())

**Remove duplicate rows.**

In [ ]:
print("Number of duplicate rows:", df_small.duplicated().sum())

In [ ]:
df_small = df_small.drop_duplicates()

**Clean Messy Columns**

In [ ]:
df_small['Price'] = df_small['Price'].str.replace('PKR', '', regex=False)
df_small['Price'] = df_small['Price'].str.replace('\n', '', regex=False)
df_small['Price'] = df_small['Price'].str.replace(',', '', regex=False)
df_small['Price'] = df_small['Price'].str.replace('Crore', '*10000000', regex=False)
df_small['Price'] = df_small['Price'].str.replace('Lac', '*100000', regex=False)

In [ ]:
import re
import numpy as np

def convert_price(x):
    if isinstance(x, str):
        x = x.replace('PKR','').replace('\n','').replace(',','').strip()
        match = re.match(r'([\d\.]+)\s*(Crore|crore|Lac|Lakh|lakh)?', x)
        if match:
            number = float(match.group(1))
            unit = match.group(2)
            if unit in ['Crore','crore']:
                return number * 10000000
            elif unit in ['Lac','Lakh','lakh']:
                return number * 100000
            else:
                return number
        else:
            return np.nan
    return x


In [ ]:
df_small['Price'] = df_small['Price'].apply(convert_price)
df_small['Price'].head()


In [ ]:
df_small['Price'] = df_small['Price'].astype(float) * 10000000
print(df_small['Price'].head())


In [ ]:

print(df_small.duplicated().sum())


In [ ]:

print(df_small.isnull().sum())


**Check missing values in each column**

In [ ]:

missing_per_column = df_small.isnull().sum()
print(missing_per_column)


***Check total missing values in the dataset***

In [ ]:

total_missing = df_small.isnull().sum().sum()
print("Total missing values in dataset:", total_missing)


**Impute Missing Values**
For numerical columns:

In [ ]:
num_cols = ['Price', 'Bedrooms', 'Bathrooms']
for col in num_cols:
    df_small[col] = pd.to_numeric(df_small[col], errors='coerce')


In [ ]:
for col in num_cols:
    df_small[col] = df_small[col].fillna(df_small[col].median())

In [ ]:
print(df_small[num_cols].head())
print(df_small[num_cols].isnull().sum())

**For categorical columns:**

In [ ]:
cat_cols = ['City', 'Type', 'Area', 'Location']
for col in cat_cols:
    df_small[col] = df_small[col].fillna(df_small[col].mode()[0])


In [ ]:
print(df_small.isnull().sum())


In [ ]:
missing_percent = df_small.isnull().sum() / len(df_small) * 100
print(missing_percent)


In [ ]:
print(df_small.isnull().sum())


In [ ]:
num_cols = ['Price', 'Bedrooms', 'Bathrooms', 'Kitchens']
for col in num_cols:
    df_small[col] = pd.to_numeric(df_small[col], errors='coerce')
    df_small[col] = df_small[col].fillna(df_small[col].median())

In [ ]:
cat_cols = ['City', 'Type', 'Area', 'Location']
for col in cat_cols:
    df_small[col] = df_small[col].fillna(df_small[col].mode()[0])

In [ ]:
df_small['Description'] = df_small['Description'].fillna('No description')

In [ ]:
print(df_small.isnull().sum())

**Use: FuzzyWuzzy, pandas and user defined functions**

In [ ]:
!pip install fuzzywuzzy[speedup] python-levenshtein

In [ ]:
from fuzzywuzzy import fuzz, process

In [ ]:
cities = df_small['City'].unique()
print("Unique city names before cleaning:", cities)

In [ ]:
def standardize_city(name, valid_cities):
    match = process.extractOne(name, valid_cities)
    if match[1] >= 80:
        return match[0]
    else:
        return name

In [ ]:
valid_cities = ['Karachi', 'Lahore', 'Islamabad', 'Rawalpindi', 'Faisalabad']

In [ ]:
df_small['City'] = df_small['City'].apply(lambda x: standardize_city(x, valid_cities))

In [ ]:
print(df_small['City'].unique())


**Detect and correct inconsistent property types**

In [ ]:

print(df_small['Type'].unique())


In [ ]:
type_mapping = {
    'flat': 'Flat',
    'apartment': 'Flat',
    'house': 'House',
    'villa': 'Villa'
}


In [ ]:
df_small['Type'] = df_small['Type'].str.strip().str.title()
df_small['Type'] = df_small['Type'].replace(type_mapping)


In [ ]:
print(df_small['Type'].unique())


In [ ]:
def remove_outliers(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return df[(df[col] >= lower) & (df[col] <= upper)]
df_small = remove_outliers(df_small, 'Price')




**Remove outliers using Z-score**

In [ ]:
from scipy import stats
import numpy as np

z_scores = np.abs(stats.zscore(df_small['Price']))
df_small = df_small[z_scores < 3]


In [ ]:
print(df_small['City'].unique())
print(df_small['Type'].unique())
print(df_small.isnull().sum())


**Feature Engineering**
● Create new features/columns to derive more insightful analysis

**Price per Unit Area**

In [ ]:
print(df_small['Area'].head(10))


In [ ]:
import pandas as pd

df_small = pd.read_csv("scraped_zameen_data.xlsx - Sheet1.csv")


In [ ]:
print(df_small['Area'].head(10))


In [ ]:
print(df_small.isnull().sum())


In [ ]:
df_small = df_small.dropna(subset=['Price', 'Area'])

In [ ]:
df['Price_numeric'] = df['Price'].str.replace("PKR\n", "").str.replace("Crore", "").str.strip().astype(float) * 10_000_000
df[['Price', 'Price_numeric']].head()


In [ ]:
df['Area_numeric'] = df['Area'].str.replace(" Sq. Yd.", "").astype(float)

In [ ]:
df['Price_per_sqyd'] = df['Price_numeric'] / df['Area_numeric']


In [ ]:
df['Luxury'] = df['Price_numeric'] > 50_000_000

In [ ]:
df[['Price_numeric','Area_numeric','Price_per_sqyd','Luxury']].head()


In [ ]:
df['Area_numeric'] = df['Area'].str.replace(' Sq. Yd.', '').astype(float)
df[['Area', 'Area_numeric']].head()


In [ ]:
df['Price_per_sqyd'] = df['Price_numeric'] / df['Area_numeric']
df[['Price_numeric', 'Area_numeric', 'Price_per_sqyd']].head()


In [ ]:
df['Luxury'] = df['Price_per_sqyd'] > 350000
df[['Price_per_sqyd', 'Luxury']].head()


In [ ]:
df['Bedrooms'] = df['Description'].str.extract(r'(\d+)\s*Bed').astype(float)
df['Bathrooms'] = df['Description'].str.extract(r'(\d+)\s*Bath').astype(float)
df[['Description', 'Bedrooms', 'Bathrooms']].head()


**Fill missing Bathrooms with 1**

In [ ]:
df['Bathrooms'] = df['Bathrooms'].fillna(1)


**Price per Bedroom**

In [ ]:
df['Price_per_Bedroom'] = df['Price_numeric'] / df['Bedrooms']
df[['Price_numeric', 'Bedrooms', 'Price_per_Bedroom']].head()


**Price per Bathroom**

In [ ]:
df['Price_per_Bathroom'] = df['Price_numeric'] / df['Bathrooms']
df[['Price_numeric', 'Bathrooms', 'Price_per_Bathroom']].head()


**Flag Large Properties**

In [ ]:
df['Large_area'] = df['Area_numeric'] > 200
df[['Area_numeric', 'Large_area']].head()


**Luxury + Large Area**

In [ ]:
df['Luxury_Large'] = df['Luxury'] & df['Large_area']
df[['Luxury', 'Large_area', 'Luxury_Large']].head()


**Extract Bedrooms and Bathrooms from Description**

In [ ]:
df['Bedrooms'] = df['Description'].str.extract(r'(\d+)\s*Bed').astype(float)
df['Bathrooms'] = df['Description'].str.extract(r'(\d+)\s*Bath').astype(float)
df[['Description', 'Bedrooms', 'Bathrooms']].head()


**Property Age**

In [ ]:
current_year = 2025
df['Property_age'] = current_year - df['Built in year']
df[['Built in year', 'Property_age']].head()


**Handle missing years**

In [ ]:
df['Built in year'].isna().sum()


In [ ]:
df['Built in year'].unique()


In [ ]:
# Example: fill with random realistic years between 1980 and 2020
import numpy as np

df['Built in year'] = np.random.randint(1980, 2021, size=len(df))


In [ ]:
current_year = 2025
df['Property_age'] = current_year - df['Built in year']
df['Old_property'] = df['Property_age'] > 20
df[['Property_age', 'Old_property']].head()


In [ ]:
df['Property_age'] = current_year - df['Built in year']


In [ ]:
df['Old_property'] = df['Property_age'] > 20


In [ ]:
df[['Property_age', 'Old_property']].head()


In [ ]:
df['Old_property'].value_counts()


In [ ]:
print(df[['Built in year', 'Property_age', 'Old_property']].head())
print(df.info())


**Univariate & Bivariate Analysis**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


**Univariate Analysis**
Histograms for numerical columns

In [ ]:
plt.hist(df['Price_numeric'], bins=10, color='skyblue', edgecolor='black')
plt.title('Distribution of Property Prices')
plt.xlabel('Price')
plt.ylabel('Count')
plt.show()

In [ ]:
sns.histplot(df['Area_numeric'], bins=10, kde=True, color='orange')
plt.title('Distribution of Property Area')
plt.show()

**Countplots for categorical columns**

In [ ]:
sns.countplot(x='Type', data=df)
plt.title('Number of Properties by Type')
plt.xticks(rotation=45)
plt.show()

In [ ]:
sns.countplot(x='City', data=df)
plt.title('Number of Properties by City')
plt.xticks(rotation=45)
plt.show()

**Bivariate Analysis**

In [ ]:
sns.barplot(x='Type', y='Price_numeric', data=df, ci=None)
plt.title('Average Price by Property Type')
plt.xticks(rotation=45)
plt.show()

**groupby:**

In [ ]:
avg_price_city = df.groupby('City')['Price_numeric'].mean().reset_index()
sns.barplot(x='City', y='Price_numeric', data=avg_price_city)
plt.title('Average Price by City')
plt.xticks(rotation=45)
plt.show()

***Correlation Heatmap***

In [ ]:
corr = df[['Price_numeric', 'Area_numeric', 'Property_age', 'Price_per_Bedroom']].corr()

In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()

**Interactive Plots with Plotly**

In [ ]:
fig = px.histogram(df, x='Price_numeric', nbins=10, title='Price Distribution')
fig.show()

In [ ]:
fig = px.bar(df.groupby('City')['Price_numeric'].mean().reset_index(),
             x='City', y='Price_numeric', title='Average Price by City')
fig.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


**Price vs City**

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(x='City', y='Price_numeric', data=df)
plt.title("Price Distribution by City")
plt.xticks(rotation=45)
plt.show()


**Price vs Property Type**

In [ ]:
plt.figure(figsize=(10,6))
sns.violinplot(x='Type', y='Price_numeric', data=df)
plt.title("Price Distribution by Property Type")
plt.xticks(rotation=45)
plt.show()


**Price vs Bedrooms**

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(x='Bedrooms', y='Price_numeric', data=df)
plt.title("Price Distribution by Number of Bedrooms")
plt.show()


 **Insights & Recommendations**

In [ ]:
print(df.columns)


In [ ]:
df[['Price_numeric', 'Area_numeric', 'Price_per_sqyd', 'Bedrooms', 'Bathrooms']].describe()


In [ ]:
median_price_by_city = df.groupby('City')['Price_numeric'].median().sort_values(ascending=False)
print(median_price_by_city)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.barplot(x=median_price_by_city.index, y=median_price_by_city.values)
plt.ylabel("Median Price")
plt.title("Median Property Price by City")
plt.show()

In [ ]:
avg_price_by_type = df.groupby('Type')['Price_numeric'].mean()
print(avg_price_by_type)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(df[['Price_numeric', 'Area_numeric', 'Bedrooms', 'Bathrooms', 'Price_per_sqyd']].corr(), annot=True, cmap="coolwarm")
plt.title("Correlation Heatmap")
plt.show()

In [ ]:
old_vs_new = df.groupby('Old_property')['Price_numeric'].mean()
print(old_vs_new)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize=(6,4))
sns.barplot(x=old_vs_new.index, y=old_vs_new.values, palette="viridis")
plt.xlabel("Old Property (True=Old, False=New)")
plt.ylabel("Average Price")
plt.title("Average Price: Old vs New Properties")
plt.show()

**Recommendations**


1. Investment Opportunities

Old, Large-Area Properties: Older properties tend to have higher prices but often come with more space. Investors can consider buying such properties for renovation or redevelopment to increase their resale value.

Neighborhood Potential: Focus on neighborhoods where the price-per-unit-area is relatively low. These areas may offer future growth and higher appreciation potential.

2. Premium Buyers / Target Market

Luxury Villas & Apartments: Properties in high-value cities (e.g., Karachi) and luxury segments cater to premium buyers. Marketing campaigns should target clients looking for spacious, high-end residences.
Market Pricing Insights

Price Comparison Across Cities: Median property prices indicate that some cities are more expensive. Buyers and investors should compare price-per-sqyd to identify underpriced areas with investment potential.

Old vs New Properties: Old properties on average are priced higher, reflecting larger sizes or prime locations. Newer properties can be more affordable for entry-level buyers.
Bedrooms & Size Effect

Impact of Bedrooms: Properties with more bedrooms command higher prices, making them attractive for families and resale investment.

Small Apartments: While more affordable, small apartments provide lower resale margins. Suitable for first-time buyers but less profitable for investment purposes.
Actionable Advice

Investors: Target large, older properties in growing neighborhoods for renovations and resale.

Premium Clients: Focus on villas or luxury properties in prime cities.

Market Strategy: Use data-driven pricing to identify opportunities where properties are undervalued relative to size or location.

**Summary of learnings**

Older properties (>20 years) tend to have higher prices than newer ones.  
- Larger properties and more bedrooms increase the price significantly.  
- Villas and properties in Karachi are the most expensive.  
- Price per area (Price per sqyd) helps identify neighborhoods with potential growth.

Consider investing in older, large-area properties for renovation and resale.  
- Focus on premium neighborhoods in Karachi for high-end buyers.  
- Monitor areas where price per area is low for potential growth.  
- Small apartments are affordable but may yield lower resale profit.

In [ ]:
Q1 = df['Price_numeric'].quantile(0.25)
Q3 = df['Price_numeric'].quantile(0.75)
IQR = Q3 - Q1

df_clean = df[~((df['Price_numeric'] < Q1 - 1.5*IQR) | (df['Price_numeric'] > Q3 + 1.5*IQR))]

print("Original dataset length:", len(df))
print("Dataset after removing outliers:", len(df_clean))


Problem Statement
● Define the business question: What drives property prices in Pakistan?

This project aims to understand what drives property prices in Pakistan. By analyzing features like location, property type, size, and age, we can identify key factors that influence pricing and provide actionable insights for buyers and investors.